# King County Real Estate Module

In [2]:
""" one line summary.

overall description of the module or program.  Optionally, it may also
contain a brief description of exported classes and functions and/or usage
examples.

    Typical Usage Example:
    
    foo = ClassFoo()
    bar = foo.FunctionBar()
"""

# Import packages
import requests
import io
import os

import numpy as np
import pandas as pd

import datetime
import time


# Define functions
def get_county_data(file_name, num_rows=None):
    """ Retrieves a single data-file from the King County Assessors webstie.
    Retrieves the single data-file from the King County Assessors webstie
    defined by file_name using the Pandas read_csv() function
    Args:
        file_name: The name of the file to download.
                   Spaces will be properly formatted
        num_rows: The number of rows to return.
    Returns:
        A Pandas dataframe containing all columns of the data retreived from
        the King County Assessor's webstie and number of rows equal to
        num_rows (defaults to all).
    Raises:
        ValueError: If passed file_name is not a string.
        ValueError: If passed file_name is not valid.
        ValueError: If passed num_rows is not a positive integer.
        OSError: If a connection to the URL is unable to be established.
    """

    # Initialize dataframe
    data_raw = pd.DataFrame()

    # Check inputs
    valid_names = ['Accessory', 'Apartment%20Complex', 'Change%20History',
                   'Change%20History%20Detail', 'Commercial%20Building',
                   'Condo%20Complex%20and%20Units',
                   'District%20Levy%20Reference',
                   'Environmental%20Restriction',
                   'Home%20Improvement%20Applications',
                   'Home%20Improvement%20Exemptions', 'Legal', 'Lookup',
                   'Notes', 'Parcel', 'Permit', 'Real%20Property%20Account',
                   'Real%20Property%20Appraisal%20History',
                   'Real%20Property%20Sales', 'Residential%20Building',
                   'Review%20History', 'Tax%20Data', 'Unit%20Breakdown',
                   'Vacant%20Lot', 'Value%20History']

    if not isinstance(file_name, str):
        raise ValueError('Passed file_name must be of type string')

    file_name = file_name.replace(' ', '%20')

    if file_name not in valid_names:
        raise ValueError('The file name you\'ve entered is not valid. ' +
                         'Please check ' +
                         'https://info.kingcounty.gov/assessor/' +
                         'DataDownload/default.aspx for correct file name')

    if num_rows is not None:
        if not isinstance(num_rows, int) & (num_rows > 0):
            raise ValueError('Number or rows to return must be a positive' +
                             f'integer not {num_rows}')

    # Define base URL
    url = f'https://aqua.kingcounty.gov/extranet/assessor/{file_name}.zip'

    # Read in the data
    try:
        data_raw = pd.read_csv(url,
                               nrows=num_rows,
                               low_memory=False)

    except OSError:
        # try three more times with delay
        for i in range(3):
            time.sleep(1)
            try:
                data_raw = pd.read_csv(url,
                                       nrows=num_rows,
                                       low_memory=False)
            except OSError:
                pass
        if data_raw.empty:
            raise OSError('King County Assessor\'s page could not be ' +
                          'reached. Please check that ' +
                          'https://info.kingcounty.gov/assessor/' +
                          'DataDownload/default.aspx is available')

    except UnicodeDecodeError:
        # change encoding to latin-1 in read_csv
        data_raw = pd.read_csv(url,
                               nrows=num_rows,
                               encoding='latin-1',
                               low_memory=False)

    # Check result and return
    if data_raw.shape[0] == 0:
        raise RuntimeError('No data was returned. Please try again later.')

    return data_raw


def get_data():
    try:
        return get_data_from_Redfin()
    except ValueError:
        return get_data_from_file()


def get_data_from_file():
    current_path = os.getcwd()
    parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
    data_path = os.path.join(parent_path, "data")
    redfin_path = os.path.join(data_path,"redfin")
    file_path = os.path.join(redfin_path, "All_King_Redfin.csv")
    df = pd.read_csv(file_path)
    return df


def get_data_from_Redfin():
    all_king_url = "https://www.redfin.com/stingray/api/gis-csv?al=1&cluster_bounds=-123.04941%2046.84777%2C-121.01694%2046.84777%2C-121.01694%2047.92442%2C-123.04941%2047.92442%2C-123.04941%2046.84777&market=seattle&min_stories=1&num_homes=5000&ord=redfin-recommended-asc&page_number=1&region_id=118&region_type=5&sf=1,2,3,5,6,7&status=1&uipt=1,2,3,4,5,6&v=8"
    urlData = requests.get(all_king_url).content
    if "spam bot" in str(urlData):
        raise ValueError("Redfin api error")
    else:
        return pd.read_csv(io.StringIO(urlData.decode('utf-8')))


#def filter_county_data(zip_code: list, start_year='2010', start_month='1', start_day='1',
#                       end_year='2020', end_month='1', end_day='1'):
def filter_county_data(df_sale, df_building, df_parcel, df_lookup, 
                       zip_code: list, start_year='2010', start_month='1', start_day='1',
                       end_year='2020', end_month='1', end_day='1'):
    """ Cleans and organizes data retrieved from the King County Assessors website.
    
    Renames columns consistently, filters data using default and customizable inputs,
    merges data to a single csv file.
    
    Args:
        zip_code(list): List of zip codes in the King County.
        start_year(str): Include property sale data from this year.
        start_month(str): Include property sale data from this month.
        start_day(str): Include property sale data from this day.
        end_year(str): Include property sale data to this year.
        end_month(str): Include property sale data to this month.
        end_day(str): Include property sale data to this day.

    Returns:
        A Pandas dataframe containing all the data retrieved from
        the King County Assessor's website, filtered and merged.

    Raises:
        ValueError: If passed zip code is not valid.
        ValueError: If passed start_year is before the first record.
        ValueError: If passed end_year is after the last record.
        ValueError: If start date is after end date based on passed values.
    """
    
    ## get data using get_county_data()
    #df_sale = get_county_data("Real%20Property%20Sales")
    #df_building = get_county_data("Residential%20Building")
    #df_parcel = get_county_data("Parcel")
    #df_lookup = get_county_data("Lookup")

    df_sale = df_sale[df_sale['Major'] != '      ']
    df_sale = df_sale.astype({'Major': int, 'Minor': int})

    #df_lookup_items = pd.read_csv('../data/look_up_item.csv')
    #df_col_names = pd.read_csv('../data/column_names.csv')
    
    df_lookup_items = pd.read_csv('https://raw.githubusercontent.com/chrico7/data515_project/master/data/look_up_item.csv')
    df_col_names = pd.read_csv('https://raw.githubusercontent.com/chrico7/data515_project/master/data/column_names.csv')

    df_sale.columns = df_col_names[df_col_names['source'] == 'sale'].name.tolist()
    df_building.columns = df_col_names[df_col_names['source'] == 'building'].name.tolist()
    df_parcel.columns = df_col_names[df_col_names['source'] == 'parcel'].name.tolist()
    df_lookup.columns = df_col_names[df_col_names['source'] == 'lookup'].name.tolist()

    df_lookup['Look Up Description'] = df_lookup['Look Up Description'].str.strip()

    # get valid zip codes in King County
    kc_zip_codes = df_building['Zip code'].dropna().unique()
    index = []
    for i in range(len(kc_zip_codes)):
        if type(kc_zip_codes[i]) == float:
            kc_zip_codes[i] = int(kc_zip_codes[i])
            kc_zip_codes[i] = str(kc_zip_codes[i])
        if kc_zip_codes[i][:2] != '98' or (len(kc_zip_codes[i]) != 5 and len(kc_zip_codes[i]) != 10):
            index.append(i)
    valid_zip = np.delete(kc_zip_codes, index)
    for i in range(len(valid_zip)):
        if len(valid_zip[i]) == 10:
            valid_zip[i] = valid_zip[i][:5]

    # check zip code(s)
    for code in zip_code:
        if code not in np.unique(valid_zip):
            raise ValueError('The zip code ' + str(code) + ' you\'ve entered is not in King County')

    # check dates
    df_sale['Document Date'] = pd.to_datetime(df_sale['Document Date'])
    start_date = start_year + '-' + start_month + '-' + start_day
    end_date = end_year + '-' + end_month + '-' + end_day

    begin_year = df_sale.sort_values(['Document Date'], ascending=[True])['Document Date'].iloc[0].year
    end_year = df_sale.sort_values(['Document Date'], ascending=[True])['Document Date'].iloc[-1].year
    if int(start_year) < int(begin_year):
        raise ValueError('There is no record before year' + str(begin_year))
    if int(start_year) > int(end_year):
        raise ValueError('There is no record after year' + str(end_year))
    if datetime.date(int(start_year), int(start_month), int(start_day)) > \
            datetime.date(int(end_year), int(end_month), int(end_day)):
        raise ValueError('Start date is after end date')

    # clean up the data
    df_building['Zip code'] = pd.to_numeric(df_building['Zip code'], errors='coerce')
    df_building = df_building.dropna(subset=['Zip code'])
    df_building['Zip code'] = df_building['Zip code'].astype(int)
    df_building['Zip code'] = df_building['Zip code'].astype(str)

    # limit properties to only single family houses
    df_parcel_sf = df_parcel[df_parcel['Property Type'] == 'R']
    df_parcel_sf = df_parcel_sf.drop(columns=['Property Type'])
    df_sale_sf = df_sale[df_sale['Property Type'] == 11]
    df_building_sf = df_building[df_building['Number Living Units'] == 1]

    # filter by a start date and end date
    df_sale_sf_recent = df_sale_sf[df_sale_sf['Document Date'] >= start_date]
    df_sale_sf_recent = df_sale_sf_recent[df_sale_sf_recent['Document Date'] <= end_date]

    # filter by zip code(s)
    df_building_sf_zip = pd.DataFrame()
    for code in zip_code:
        df_building_sf_zip = df_building_sf_zip.append(df_building_sf[df_building_sf['Zip code'] == code])

    new_df = pd.merge(df_sale_sf_recent, df_building_sf_zip, how='left', left_on=['Major', 'Minor'], right_on=['Major', 'Minor'])
    df_all = pd.merge(new_df, df_parcel_sf, how='left', left_on=['Major', 'Minor'], right_on=['Major', 'Minor'])

    # replace numerical codes in records to readable descriptions
    for col in df_all.columns:
        if col in df_lookup_items['Field Name'].tolist():
            look_up_type = int(df_lookup_items[df_lookup_items['Field Name'] == col]['Look Up'])
            look_up_items = df_lookup[df_lookup['Look Up Type'] == look_up_type]
            description_list = []
            for i in range(len(df_all[col])):
                num = df_all[col].iloc[i]
                description = look_up_items[look_up_items['Look Up Item'] == num]['Look Up Description']
                if len(description) == 0:
                    description_list.append('nan')
                else:
                    description_list.append(description.values[0])
            df_all[col] = description_list
    return df_all


#if __name__ == "__main__":
#    zip_code = [str(item) for item in input("Enter zip code (separated by comma) : ").split()]
#    start_year = (input("Enter start year: "))
#    start_month = (input("Enter start month: "))
#    start_day = (input("Enter start day: "))
#    end_year = (input("Enter end year: "))
#    end_month = (input("Enter end month: "))
#    end_day = (input("Enter end day: "))
#    filter_county_data(zip_code, start_year, start_month, start_day, end_year, end_month, end_day)


In [3]:
####
## TEST MODULE
####

In [4]:
# Read in KC data
df_sale = get_county_data("Real%20Property%20Sales")
df_building = get_county_data("Residential%20Building")
df_parcel = get_county_data("Parcel")
df_lookup = get_county_data("Lookup")

In [5]:
df_sale.shape, df_building.shape, df_parcel.shape, df_lookup.shape

((2087147, 24), (515018, 50), (614827, 81), (1208, 3))

In [6]:
# Read in Redfin data
#df_redfin = get_data_from_Redfin()
df_redfin = pd.read_csv('https://raw.githubusercontent.com/chrico7/data515_project/master/data/redfin/All_King_Redfin.csv')

In [7]:
df_redfin.shape

(4140, 27)

In [10]:
# Clean KC data
kc_data = filter_county_data(df_sale, df_building, df_parcel, df_lookup,
                             zip_code = ['98105'],
                             start_year ='2019',
                             start_month ='1',
                             start_day = '1',
                             end_year = '2019', end_month = '12', end_day = '31')

C:\Users\chrico7\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\chrico7\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\chrico7\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  r

In [13]:
kc_data.shape

(27138, 150)

In [14]:
kc_data.columns

Index(['Excise Tax Number', 'Major', 'Minor', 'Document Date', 'Sale Price',
       'Recording Number', 'Recording Volume', 'Recording Page', 'Plat Number',
       'Plat Type',
       ...
       'Seismic Hazard', 'Landslide Hazard', 'Steep Slope Hazard', 'Stream',
       'Wetland', 'Species Of Concern', 'Sensitive Area Tract',
       'Water Problems', 'Transportation Concurrency', 'Other Problems'],
      dtype='object', length=150)

In [18]:
path = r'C:/Users/chrico7/Documents/__Corey Christopherson/MS Data Science/Courses/DATA 515/Final Project/'
kc_data.to_csv(r'{}sample_data_98105_2019.csv'.format(path), index=False)